
# Chatbot for E-commerce Platform with Rule-Based Responses
This notebook contains the code to train and run a chatbot for an e-commerce platform.


In [1]:

#Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from sklearn.metrics import classification_report, accuracy_score


In [2]:
# Downloading NLTK resources: tokenizers (punkt), stopwords, and WordNet lexical database

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Loading product and conversational datasets into pandas dataframes

product_df = pd.read_csv('product_data.csv')
conversational_df = pd.read_csv('conversational_data.csv')

In [5]:
#displaying the beggining part of product dataframe
product_df.head()

,ProductName,Description,Price_LKR
0,Samsung Galaxy S21,"6.2 inch, 128GB storage, 12MP Camera",190000
1,Apple iPhone 13,"6.1 inch, 128GB storage, 12MP Camera",250000
2,Dell Inspiron Laptop,"15.6 inch, 8GB RAM, 512GB SSD",160000
3,Sony WH-1000XM4 Headphones,"Over-ear, Wireless, Noise-cancelling",75000
4,Apple Watch Series 7,"GPS, 44mm, Blood Oxygen App",120000


In [6]:
#displaying the beggining part of product dataframe
conversational_df.head()

,UserQuery,Intent
0,How much is the Samsung Galaxy S21?,PriceInquiry
1,Tell me more about the Samsung Galaxy S21.,ProductDetails
2,Do you have the Samsung Galaxy S21 in stock?,ProductAvailability
3,How much is the Apple iPhone 13?,PriceInquiry
4,Tell me more about the Apple iPhone 13.,ProductDetails


In [7]:
# Initializing lemmatizer and stemmer for text preprocessing

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to get synonyms of a word using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def preprocess_text(text):
    # 1. Case Insensitivity
    text = text.lower()

    # 2. Tokenization
    tokens = word_tokenize(text)

    # 3. Stopword Removal
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # 4. Stemming
    tokens = [stemmer.stem(t) for t in tokens]

    # 5. Lemmatization
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    # 6. Handling Similar Words using WordNet
    tokens_synonyms = []
    for token in tokens:
        synonyms = get_synonyms(token)
        if synonyms:
            tokens_synonyms.extend(synonyms)
        else:
            tokens_synonyms.append(token)

    return ' '.join(tokens_synonyms)

In [8]:
# Processing the 'UserQuery' column from the conversational dataset using the 'preprocess_text' function and storing the results in a new 'ProcessedQuery' column

conversational_df['ProcessedQuery'] = conversational_df['UserQuery'].apply(preprocess_text)


In [9]:
# Splitting the data
# Splitting the processed queries and their corresponding intents into training and testing sets with an 80-20 split ratio

X_train, X_test, y_train, y_test = train_test_split(conversational_df['ProcessedQuery'], conversational_df['Intent'], test_size=0.2)

In [10]:
# Building the intent classifier
# Constructing an intent classifier using a pipeline that combines TF-IDF vectorization, feature scaling, and a linear kernel Support Vector Classifier
# then training it with the training data

intent_classifier = make_pipeline(TfidfVectorizer(), StandardScaler(with_mean=False), SVC(kernel='linear'))
intent_classifier.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('standardscaler', StandardScaler(with_mean=False)),
                ('svc', SVC(kernel='linear'))])

In [11]:
# Function to extract a product name from a given query by checking if any product name from the product dataframe exists within the query
#Then returns the product name if found, otherwise returns None


def extract_product_name(query):
    for product in product_df['ProductName']:
        if product.lower() in query.lower():
            return product
    return None

In [12]:
# Function to generate an appropriate response based on the given user query:


def generate_response(user_query):
    # 1. Directly match user queries to known questions (case-insensitive) for rows starting from 302
    matched_responses = conversational_df.iloc[301:].loc[conversational_df['UserQuery'].str.lower() == user_query.lower(), 'Intent'].values
    if matched_responses.size > 0:
        return matched_responses[0]

    # 2. Preprocess the user query
    preprocessed_user_query = preprocess_text(user_query)

    # 3. Predict the intent using the intent_classifier (It already contains the vectorizer)
    intent = intent_classifier.predict([preprocessed_user_query])[0]

    # 4. Extract product name if any
    product_name = extract_product_name(user_query)

    # 5. Generate a response based on the intent and product name
    if intent == "PriceInquiry" and product_name:
        price = product_df[product_df['ProductName'] == product_name]['Price_LKR'].values[0]
        return f"The price of the {product_name} is LKR {price}."
    elif intent == "ProductDetails" and product_name:
        description = product_df[product_df['ProductName'] == product_name]['Description'].values[0]
        return f"The {product_name} has the following features: {description}."
    elif intent == "ProductAvailability" and product_name:
        return f"Yes, we have the {product_name} in stock."
    else:
        return "I'm sorry, I couldn't understand that. Can you please rephrase or ask something else?"


In [14]:
# Testing the 'generate_response' function

user_query = "What is the Price of Apple iphone 13?"
print(generate_response(user_query))



The price of the Apple iPhone 13 is LKR 250000.


In [15]:
# Save the model to disk
import joblib

joblib.dump(intent_classifier, 'intent_classifier.pkl')

['intent_classifier.pkl']

In [16]:
# Predict intents on the test data
y_pred = intent_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")




Accuracy: 0.9402985074626866
